In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

In [2]:
# Step 1: Data Augmentation and Loading
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

dataset_dir = 'dataset'  # Replace with your dataset directory
full_dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)

# Split dataset into training (80%) and testing (20%)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [3]:
# Step 2: Load Pretrained Model and Modify It
num_classes = len(full_dataset.classes)
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Step 3: Define Loss Function, Optimizer, and Learning Rate Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

/Users/maharaja/pytorch-env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/maharaja/pytorch-env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Step 4: Training the Model
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Learning rate scheduler step
    scheduler.step()

    # Print average loss per epoch
    average_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')

# Save the trained model
model_path = 'plant_disease_resnet50.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Epoch 1/25:   0%|          | 0/516 [00:00<?, ?it/s]

Epoch 1/25: 100%|██████████| 516/516 [07:38<00:00,  1.13it/s]


Epoch [1/25], Loss: 0.6460


Epoch 2/25: 100%|██████████| 516/516 [07:38<00:00,  1.13it/s]


Epoch [2/25], Loss: 0.3611


...

Epoch [23/25], Loss: 0.0125


Epoch 24/25: 100%|██████████| 516/516 [07:38<00:00,  1.13it/s]


Epoch [24/25], Loss: 0.0120


Epoch 25/25: 100%|██████████| 516/516 [07:38<00:00,  1.12it/s]

Epoch [25/25], Loss: 0.0115
Model saved to plant_disease_resnet18.pth


In [5]:
# Step 5: Evaluate the Model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy on the test images: {accuracy:.2f}%')

Evaluating: 100%|██████████| 129/129 [01:10<00:00,  1.82it/s]

Accuracy on the test images: 98.72%


In [8]:
# Step 6: Use the Model to Predict a Single Image
def predict_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    
    class_index = predicted.item()
    class_label = full_dataset.classes[class_index]
    return class_label

# Example usage:
image_path = '1.png'  # Replace with your image path
predicted_label = predict_image(image_path)
print(f'The predicted class for the image is: {predicted_label}')

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [10]:
from sklearn.metrics import f1_score
import torch
from torchvision import models
from tqdm import tqdm

# Load the saved model with the correct output size
model = models.resnet18()
num_classes = 15  # Adjust this based on your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Modify the fully connected layer

# Load the saved model state
model.load_state_dict(torch.load('plant_disease_resnet18.pth'))
model.eval()  # Set the model to evaluation mode

true_labels = []
pred_labels = []

# Assuming you have your test_loader defined
# Disable gradient calculation for evaluation
with torch.no_grad():
    # Wrap test_loader with tqdm for progress tracking
    for inputs, labels in tqdm(test_loader, desc="Evaluating Model"):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        pred_labels.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate F1 score
f1 = f1_score(true_labels, pred_labels, average='weighted')  # or 'macro' for multi-class
print(f"F1 Score: {f1}")

/var/folders/g1/jh2048gn2tx119xcnmnxlg4m0000gn/T/ipykernel_796/3774038145.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('plant_diseas

F1 Score: 0.9881013655816576
